In [1]:
import logging
import pandas as pd
import numpy as np
from pathlib import Path
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, matthews_corrcoef
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import os
import torch

import warnings
warnings.filterwarnings('ignore')

# Set up logging
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)

c:\Users\willi\Desktop\Uni\3rd_year\NLU\comp38412-not-broken\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Path configuration
DATA_DIR = Path('../../data')
TRAIN_PATH = DATA_DIR / 'train.csv'
AUGMENTED_DATA_PATH = DATA_DIR / 'train_augmented.csv'
DEV_PATH = DATA_DIR / 'dev.csv'
MODEL_SAVE_PATH = Path('./models')
MODEL_SAVE_PATH.mkdir(parents=True, exist_ok=True)

# Optuna config
N_TRIALS = 250  # Number of Optuna trials

In [3]:
def get_device() -> torch.device:
    """Determine the device to use for computations."""
    if torch.cuda.is_available():
        return torch.device('cuda')
    elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        return torch.device('mps')
    else:
        return torch.device('cpu')

In [4]:
def prepare_data(train_df, aug_train_df, dev_df):
    """Prepare data for XGBoost training."""
    # Combine claim and evidence into a single text feature for TF-IDF
    train_df['text'] = train_df['Claim'] + " [SEP] " + train_df['Evidence']
    aug_train_df['text'] = aug_train_df['Claim'] + " [SEP] " + aug_train_df['Evidence']
    dev_df['text'] = dev_df['Claim'] + " [SEP] " + dev_df['Evidence']
    
    # Extract labels
    train_labels = train_df['label'].values
    aug_train_labels = aug_train_df['label'].values
    dev_labels = dev_df['label'].values
    
    # Combine the augmented training data with the original training data
    train_df = pd.concat([train_df, aug_train_df])
    train_labels = np.concatenate([train_labels, aug_train_labels])
    
    return train_df, dev_df, train_labels, dev_labels

In [5]:
def calculate_all_metrics(y_true, y_pred):
    """
    Calculate comprehensive evaluation metrics.
    
    Returns:
        dict: Dictionary containing all metrics
    """
    # Basic accuracy
    accuracy = accuracy_score(y_true, y_pred)
    
    # Calculate precision, recall, f1 (macro)
    macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='macro'
    )
    
    # Calculate precision, recall, f1 (weighted)
    weighted_precision, weighted_recall, weighted_f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='weighted'
    )
    
    # Matthews Correlation Coefficient
    mcc = matthews_corrcoef(y_true, y_pred)
    
    metrics = {
        'Accuracy': accuracy,
        'Macro-P': macro_precision,
        'Macro-R': macro_recall,
        'Macro-F1': macro_f1,
        'W Macro-P': weighted_precision,
        'W Macro-R': weighted_recall,
        'W Macro-F1': weighted_f1,
        'MCC': mcc
    }
    
    return metrics

In [6]:
train_df = pd.read_csv(TRAIN_PATH)
aug_train_df = pd.read_csv(AUGMENTED_DATA_PATH)
dev_df = pd.read_csv(DEV_PATH)
   
train_df, dev_df, train_labels, dev_labels = prepare_data(train_df, aug_train_df, dev_df)

In [7]:
def objective(trial):
    global train_df, dev_df, train_labels, dev_labels
    """Optuna objective function for hyperparameter optimization."""
       
    # TF-IDF vectorizer parameters
    max_features = trial.suggest_categorical("max_features", [5000, 10000, 15000, 20000])
    min_df = trial.suggest_categorical("min_df", [1, 2, 3])
    ngram_range = trial.suggest_categorical("ngram_range", [(1, 1), (1, 2), (1, 3)])
    
    # XGBoost hyperparameters
    xgb_params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'use_label_encoder': False,
        'tree_method': 'hist',
        'random_state': 42
    }
    
    # Create pipeline
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(
            max_features=max_features,
            min_df=min_df,
            ngram_range=ngram_range,
            stop_words='english'
        )),
        ('scaler', StandardScaler(with_mean=False)),  # TF-IDF matrices are sparse
        ('xgb', xgb.XGBClassifier(
            **xgb_params,
            enable_categorical=True  # Enable categorical feature support
        ))
    ])
    
    # Train model
    logging.info(f"Training XGBoost with hyperparameters: {xgb_params}")
    
    X_train = pipeline.named_steps['tfidf'].fit_transform(train_df['text'])
    X_train = pipeline.named_steps['scaler'].fit_transform(X_train)
    
    X_dev = pipeline.named_steps['tfidf'].transform(dev_df['text'])
    X_dev = pipeline.named_steps['scaler'].transform(X_dev)
    
    # Create DMatrix objects
    dtrain = xgb.DMatrix(X_train, label=train_labels)
    ddev = xgb.DMatrix(X_dev, label=dev_labels)
    
    # Train model
    model = xgb.train(
        xgb_params,
        dtrain,
        num_boost_round=xgb_params['n_estimators'],
        evals=[(ddev, 'eval')],
        early_stopping_rounds=50,
        verbose_eval=False
    )
    
    # Make predictions
    dev_preds = (model.predict(ddev) >= 0.5).astype(int)    
    metrics = calculate_all_metrics(dev_labels, dev_preds)
    
    # Report intermediate values for pruning
    trial.report(metrics['W Macro-F1'], step=model.best_iteration)
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()
    
    return metrics['W Macro-F1']

In [8]:
if __name__ == "__main__":
    print("\nHYPERPARAMETER TUNING")
    print("=====================")
    print(f"Running {N_TRIALS} trials...")
    
    # Check if GPU is available
    device = get_device()
    if torch.cuda.is_available():
        # Enable GPU acceleration for XGBoost
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    logging.info(f"Using device: {device}")
    
    # Create a study with TPE sampler and MedianPruner
    sampler = TPESampler(seed=42, 
                         n_startup_trials=int(N_TRIALS / 10), # First 10% of trials are random, then TPE
                         multivariate=True, 
                         constant_liar=True) # constant_liar = True as we are doing distributed optimisation

    pruner = MedianPruner(n_startup_trials=5, 
                          n_warmup_steps=5, 
                          interval_steps=2)
    
    study = optuna.create_study(
        direction='maximize',
        sampler=sampler,
        pruner=pruner,
        study_name='xgboost_evidence_detection'
    )
    
    try:
        study.optimize(objective, n_trials=N_TRIALS, n_jobs=-1)
    except KeyboardInterrupt:
        print("Hyperparameter tuning interrupted.")
    
    print("\nBest trial:")
    trial = study.best_trial
    print(f"  Value (Accuracy): {trial.value}")
    print("  Params:")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

2025-03-27 00:52:47 - Using device: cpu
[I 2025-03-27 00:52:47,666] A new study created in memory with name: xgboost_evidence_detection
2025-03-27 00:52:47 - Training XGBoost with hyperparameters: {'max_depth': 6, 'learning_rate': 0.016547525297216994, 'n_estimators': 711, 'min_child_weight': 6, 'gamma': 0.013809300275517213, 'subsample': 0.5733859310769817, 'colsample_bytree': 0.8005588642947284, 'reg_alpha': 8.52189309264861e-07, 'reg_lambda': 1.5011692299546617e-05, 'objective': 'binary:logistic', 'eval_metric': 'logloss', 'use_label_encoder': False, 'tree_method': 'hist', 'random_state': 42}
2025-03-27 00:52:47 - Training XGBoost with hyperparameters: {'max_depth': 9, 'learning_rate': 0.0004456395167515201, 'n_estimators': 716, 'min_child_weight': 6, 'gamma': 5.970394445064238e-05, 'subsample': 0.735138285355728, 'colsample_bytree': 0.5270046122183482, 'reg_alpha': 0.4160180172541676, 'reg_lambda': 0.2817548729085884, 'objective': 'binary:logistic', 'eval_metric': 'logloss', 'use_l


HYPERPARAMETER TUNING
Running 250 trials...


2025-03-27 00:52:47 - Training XGBoost with hyperparameters: {'max_depth': 4, 'learning_rate': 0.016334019837592337, 'n_estimators': 490, 'min_child_weight': 7, 'gamma': 3.467101941954019e-06, 'subsample': 0.8583937660731453, 'colsample_bytree': 0.8466927003922581, 'reg_alpha': 2.6984429513126126e-07, 'reg_lambda': 0.9503974866493724, 'objective': 'binary:logistic', 'eval_metric': 'logloss', 'use_label_encoder': False, 'tree_method': 'hist', 'random_state': 42}
2025-03-27 00:52:47 - Training XGBoost with hyperparameters: {'max_depth': 10, 'learning_rate': 0.000779345446518246, 'n_estimators': 326, 'min_child_weight': 4, 'gamma': 0.00033231891040756077, 'subsample': 0.9024656845814256, 'colsample_bytree': 0.9477942946762944, 'reg_alpha': 0.07234023800813709, 'reg_lambda': 0.23729575273237563, 'objective': 'binary:logistic', 'eval_metric': 'logloss', 'use_label_encoder': False, 'tree_method': 'hist', 'random_state': 42}
2025-03-27 00:52:47 - Training XGBoost with hyperparameters: {'max_d


Best trial:
  Value (Accuracy): 0.796359627026355
  Params:
    max_features: 5000
    min_df: 3
    ngram_range: (1, 3)
    max_depth: 8
    learning_rate: 0.22260998418525665
    n_estimators: 666
    min_child_weight: 2
    gamma: 0.005341979637780388
    subsample: 0.9900049863839708
    colsample_bytree: 0.5918673659371315
    reg_alpha: 6.834558057096682e-08
    reg_lambda: 0.008776003604199037


In [ ]:
best_f1 = 0.796359627026355
best_params = {
    "max_features": 5000,
    "min_df": 3,
    "ngram_range": (1, 3),
    "max_depth": 8,
    "learning_rate": 0.22260998418525665,
    "n_estimators": 666,
    "min_child_weight": 2,
    "gamma": 0.005341979637780388,
    "subsample": 0.9900049863839708,
    "colsample_bytree": 0.5918673659371315,
    "reg_alpha": 6.834558057096682e-08,
    "reg_lambda": 0.008776003604199037,
}